In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from preprocess_and_feature_engineering import feature_engineering, preprocess 

In [2]:
# Read data sets
X_train = pd.read_csv('ais_train.csv', sep='|')
X_test = pd.read_csv('ais_test.csv')

In [3]:
X_train,X_test = feature_engineering(X_train,X_test)

In [4]:
X_train.head()

,cog,sog,rot,heading,is_moored,latitude_lag1,longitude_lag1,navstat,latitude,longitude,...,etaMonth,etaDay,etaHour,etaMinute,timeYear,timeMonth,timeDay,timeHour,timeMinute,timeSecond
0,284.0,0.7,0,88,0,NaN,NaN,0,-34.74370,-57.85130,...,1,9,23,0,24,1,1,0,0,25
1,109.6,0.0,-6,347,0,-34.74370,-57.85130,1,8.89440,-79.47939,...,12,29,20,0,24,1,1,0,0,36
2,111.0,11.0,0,112,0,8.89440,-79.47939,0,39.19065,-76.47567,...,1,2,9,0,24,1,1,0,1,45
3,96.4,0.0,0,142,0,39.19065,-76.47567,1,-34.41189,151.02067,...,12,31,20,0,24,1,1,0,3,11
4,214.0,19.7,0,215,0,-34.41189,151.02067,0,35.88379,-5.91636,...,1,25,12,0,24,1,1,0,3,51


In [5]:
X_test.head()

,ID,vesselId,scaling_factor,timeYear,timeMonth,timeDay,timeHour,timeMinute,timeSecond
0,0,412,0.3,24,05,08,00,03,16
1,1,373,0.3,24,05,08,00,06,17
2,2,181,0.3,24,05,08,00,10,02
3,3,8,0.3,24,05,08,00,10,34
4,4,65,0.3,24,05,08,00,12,27


In [6]:
# Create the trianing inputs
y_train = X_train[['longitude', 'latitude']]
X_train.drop(columns=['longitude', 'latitude'],inplace=True)

In [7]:
# Initialize the model
#rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
#rf.fit(X_train, y_train)